In [34]:
# Importing basic packages
import os
import warnings
warnings.filterwarnings('ignore')

# to load env variables
from dotenv import load_dotenv
from openai import OpenAI # OpenAI llm
from agents import Agent, Runner, trace, Handoff, OpenAIChatCompletionsModel # OpenAI SDK primitives
from openai.types.responses import ResponseTextDeltaEvent

from openai import AsyncOpenAI

# asynio
import asyncio

# Decorator Packages
from IPython.display import display, Markdown
from typing import Dict, List


In [20]:
load_dotenv(override = True)

True

In [32]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Let's create three agents 

- We need to define instructions what the agent do - you can think of system prompt
- Call openai sdk agent by defining name, instructions and model

In [21]:
instructions1  = "You are a sales agent working for Unilever, a company that produces wide range of products related to health care, deodrants & fagrance, icecream, laundry etc., . \
    You write professional, serious cold emails"

instructions2 = "You are a humorous sales agent working for Unilever, a company that produces wide range of products related to health care, deodrants & fagrance, icecream, laundry etc., . \
    You write witty, engaging cold emails that are likely to get a response"

instructions3 = "You are a humorous sales agent working for Unilever, a company that produces wide range of products related to health care, deodrants & fagrance, icecream, laundry etc., . \
    You write concise, to the point cold emails"


In [27]:
# Professional Sales Agent
sales_agent1 = Agent(name = "Professional Sales Agent", 
      instructions = instructions1,
      model = "gpt-4o-mini")


# Engaging Sales Agent
sales_agent2 = Agent(name = "Engaging Sales Agent", 
                     instructions = instructions2,
                     model = "gpt-4o-mini")

# Busy Sales Agent
sales_agent3 = Agent(name = "Busy Sales Agent", 
                     instructions = instructions3,
                     model = "gpt-4o-mini")

In [25]:
result = Runner.run_streamed(sales_agent1, input = "Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end = "", flush = True)

Subject: Elevate Your Product Offering with Unilever’s Trusted Brands

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I’m reaching out to introduce you to Unilever’s diverse portfolio of products that have been trusted by consumers worldwide for their quality and innovation.

At Unilever, we pride ourselves on delivering solutions across various categories, including health care, personal care, laundry, and food. Our commitment to sustainability and consumer satisfaction drives us to continuously enhance our offerings, ensuring that we meet the evolving needs of the market.

I believe your business could greatly benefit from adding our products to your lineup. For example, our [specific product or category] has seen remarkable success and could resonate well with your customer base.

Would you be open to a brief conversation next week to discuss how a partnership with Unilever could enhance your product offerings and drive sales? I would be ha

In [28]:
message = "Write a cold sales email"

with trace("Parallel cold emails"): # Trace helps you record the progress of the calls in 
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message)
    )

# 

In [30]:
outputs = [result.final_output for result in results] # results holds list of response from different llms

for output in outputs:
    print(output + "\n\n")

Subject: Elevate Your Offerings with Unilever's Premium Products

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I am a sales representative at Unilever, a global leader in consumer goods with a diverse portfolio spanning health care, personal care, household products, and more.

We recognize that [Recipient's Company Name] is committed to delivering quality and value to your customers. I would like to explore how our premium products can enhance your offerings and drive sales for your business.

Our range includes:

- **Personal Care**: From trusted deodorants to luxurious fragrances, we have solutions that cater to diverse consumer preferences.
- **Health Care**: Our innovative products support wellness and self-care in today’s fast-paced world.
- **Ice Cream**: Our delightful range of ice creams appeals to all age groups, providing cake and indulgence that drives repeat purchases.
- **Laundry**: Our effective detergents simplify laundry whi

## Sales Picker Agent

- The job of sales picker agent is to identify best cold emails and select that
- Now, we are going to use Gemini model as the evaluator

In [36]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/" # url endpoint

gemini_client = AsyncOpenAI(base_url = GEMINI_BASE_URL, api_key = GOOGLE_API_KEY)
gemini_model = OpenAIChatCompletionsModel(model = "gemini-2.0-flash", openai_client = gemini_client)

In [37]:
instructions4 = "You pick the best cold sales email from the given options. \
    Image you are customer and pick the oone you are most likely to respond to. \
        Do not give an explanation; reply with the selected email only"

sales_picker_agent = Agent(
    name = "Sales Picker Agent", 
    instructions = instructions4,
    model = gemini_model
)

In [ ]:
input_message = "Write a cold email message"
with trace("Sales_Picker_Response"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, input_message),
        Runner.run(sales_agent2, input_message),
        Runner.run(sales_agent3, input_message)
    )

    outputs = [result.final_output for result in results]

    # Append the output of each response of individual agent 
    emails = "Cold email messages : \n\n".join(outputs)    
    
